# Advanced dynamic seq2seq with TensorFlow

https://github.com/ematvey/tensorflow-seq2seq-tutorials/blob/master/2-seq2seq-advanced.ipynb

Encoder は今双方向です．デコーダーは `tf.nn.raw_rnn` を利用して実装されています．
トレーニング中に以前に生成されたトークンを、ターゲットシーケンスの代わりに入力としてフィードします。

In [1]:
import numpy as np
import tensorflow as tf
import helpers

tf.reset_default_graph()
sess = tf.InteractiveSession()

In [2]:
tf.__version__

'1.8.0-rc0'

In [3]:
PAD = 0
EOS = 1

vacab_size = 10
input_embedding_size = 20

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units * 2

In [4]:
encoder_inputs = tf.placeholder(
    shape=(None, None), dtype=tf.int32, name='encoder_inputs'
)
encoder_inputs_length = tf.placeholder(shape=(None, ), dtype=tf.int32, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

## Embeddings

Embedding の設定

In [5]:
embeddings = tf.Variable(tf.random_uniform([vacab_size, input_embedding_size], -1.0, 1.0),
                        dtype=tf.float32)

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

## Encoder

`tf.nn.dynamic_rnn` を使うのをやめて，`tf.nn.bidirectional_dynamic_rnn` をエンコーダーに使うことにします

In [7]:
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

In [8]:
encoder_cell = LSTMCell(encoder_hidden_units)

In [9]:
(encoder_fw_outputs, encoder_bw_outputs), (encoder_fw_final_state, encoder_bw_final_state) =  \
    tf.nn.bidirectional_dynamic_rnn(
        cell_fw=encoder_cell,
        cell_bw=encoder_cell,
        inputs=encoder_inputs_embedded,
        sequence_length=encoder_inputs_length,
        dtype=tf.float32,
        time_major=True
    )

In [10]:
encoder_fw_outputs

<tf.Tensor 'bidirectional_rnn/fw/fw/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 20) dtype=float32>

In [11]:
encoder_bw_outputs

<tf.Tensor 'ReverseSequence:0' shape=(?, ?, 20) dtype=float32>

In [12]:
encoder_fw_final_state

LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 20) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_4:0' shape=(?, 20) dtype=float32>)

In [13]:
encoder_bw_final_state

LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 20) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_4:0' shape=(?, 20) dtype=float32>)

In [14]:
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

## Decoder

In [15]:
decoder_cell = LSTMCell(decoder_hidden_units)

時間とバッチの次元は動的に変化する事ができます．つまり，実行時にバッチごとに違う設定ができるということです．

In [16]:
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

次に私たちはデコーダーを実行する長さかを決める必要があります．止める指標について幾つかのオプションが用意されています．

* 指定した展開ステップ数を超えたら停止
* モデルが生成されたら停止

翻訳のようなタスクでは，`len(encoder_input)+10` ステップの後に停止するような選択が一般的で，これは入力より少し長い翻訳を許可するためです．ここでは + 2 にしておきましょう．

In [19]:
decoder_lengths = encoder_inputs_length + 3 # うち 1 つは <EOS> のため

## Output projection